In [56]:
import pandas as pd
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
nltk.download()


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
df = pd.read_csv('urls_pos_1.txt_df.csv')

In [4]:
df.head(5)

,Unnamed: 0,usernames,ratings,reviews,usefulness_rating,name,id
0,0,zardoz-13,1,"""Hoods"" doesn't deliver the goods. This half-b...",0.500000,Hoods,tt0166219
1,1,MovieMan-112,-1,No review,-1.000000,Hoods,tt0166219
2,2,wheels-2,-1,is forced to conform to 90 minutes TV time. Ma...,0.333333,Hoods,tt0166219
3,3,Gibbs52,-1,I just called my brother Paulie on the phone a...,0.125000,Hoods,tt0166219
4,4,wmorrow59,-1,No review,-1.000000,The Goat,tt0012224


In [16]:
oceans_12 = df[df['name']=="Ocean's Twelve"]

In [34]:
review = oceans_12.sort_values('ratings', ascending=False).head(1).iloc[0]['reviews']

In [ ]:
review_negative = oceans_12.sort_values('ratings')
review_negative = review_negative[review_negative['reviews'] is not "No Review" and review_negative['ratings'] > 0].iloc[0]['reviews']
print(review_negative)

Approach 1 : VADER sentiment analysis (pretrained sentiment analyzer)

In [52]:
review_root = review.lower().replace("'", '').replace('[^\w\s]', ' ').replace(" \d+", " ").replace(' +', ' ').strip()
review_negative_root = review_negative.lower().replace("'", '').replace(
    '[^\w\s]', ' ').replace(" \d+", " ").replace(' +', ' ').strip()


In [54]:
analyser = SentimentIntensityAnalyzer()
sentiment_score_pos = analyser.polarity_scores(review_root)
print(sentiment_score_pos)
sentiment_score_neg = analyser.polarity_scores(review_negative_root)
print(sentiment_score_neg)


{'neg': 0.045, 'neu': 0.734, 'pos': 0.221, 'compound': 0.9947}
{'neg': 0.146, 'neu': 0.741, 'pos': 0.113, 'compound': -0.8202}


Stopwords removing and tokenization

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
tokenized_pos = [w for w in nltk.word_tokenize(review_root) if w not in stopwords]
tokenized_negative = [w for w in nltk.word_tokenize(
    review_negative_root) if w not in stopwords]
sentiment_score_pos2 = analyser.polarity_scores(' '.join(tokenized_pos))
sentiment_score_neg2 = analyser.polarity_scores(' '.join(tokenized_negative))
print(sentiment_score_pos2)
print(sentiment_score_neg2)
tokenized_negative


Word lemmatization + above

In [68]:
lem = WordNetLemmatizer()
lem_neg = [lem.lemmatize(w) for w in tokenized_negative]
lem_pos = [lem.lemmatize(w) for w in tokenized_pos]
sentiment_score_pos3 = analyser.polarity_scores(' '.join(lem_pos))
sentiment_score_neg3 = analyser.polarity_scores(' '.join(lem_neg))
print(sentiment_score_pos3)
print(sentiment_score_neg3)


{'neg': 0.054, 'neu': 0.709, 'pos': 0.237, 'compound': 0.9874}
{'neg': 0.237, 'neu': 0.601, 'pos': 0.162, 'compound': -0.8592}


In [70]:
o12 = oceans_12['reviews_clean'] = df['reviews'].str.lower().str.replace("'", '').str.replace(
    '[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()
#strip of punctuation, whitespace in string, and lowercase

C:\Users\doanl\AppData\Local\Temp\ipykernel_10964\2998188291.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  o12 = oceans_12['reviews_clean'] = df['reviews'].str.lower().str.replace("'", '').str.replace(
C:\Users\doanl\AppData\Local\Temp\ipykernel_10964\2998188291.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  '[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()
C:\Users\doanl\AppData\Local\Temp\ipykernel_10964\2998188291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o12 = oceans_12['reviews_clean'] = df['reviews'].str.lower().str.replace("'", '').str.replace(


In [71]:
o12.to_csv('oceans_12Reviews')